In [1]:
import argparse
import os
import random

import torch
import torch.nn as nn

import tarfile

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time

try:
    torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.deterministic = True
except:
    pass

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# If you encounter some issues regarding cuda device, e.g., "RuntimeError: CUDA Out of memory error",
# try to switch the device to cpu by using the following code

# device = torch.device('cpu')

print('Device:', device)


!pip install onedrivedownloader



Device: cuda:0
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
with tarfile.open('./train.tar', 'r') as td:
    td.extractall('./miniImagenet')

In [2]:
# Download and preprocess MiniImageNet dataset
transform = transforms.Compose([
   # transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Adjust the path to your MiniImageNet dataset directory
miniimagenet_path = './miniImagenet/train'
dataset = datasets.ImageFolder(miniimagenet_path, transform=transform)


In [ ]:

# Split the dataset

train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

print('total number of total set: {}'.format(len(dataset)))
print('total number of training set: {}'.format(len(train_set)))
print('total number of val set: {}'.format(len(val_set)))
print('total number of test set: {}'.format(len(test_set)))

# Create data loaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [3]:
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(dataset, test_size=0.2)
val_set, test_set = train_test_split(test_set, test_size=0.5)

print('total number of total set: {}'.format(len(dataset)))
print('total number of training set: {}'.format(len(train_set)))
print('total number of test set: {}'.format(len(test_set)))
print('total number of val set: {}'.format(len(val_set)))

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


total number of total set: 38400
total number of training set: 30720
total number of test set: 3840
total number of val set: 3840


In [4]:
# Create data loaders
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [5]:
num_classes = len(dataset.classes)
net = torchvision.models.resnet18(pretrained=False)
net.fc = nn.Linear(net.fc.in_features, num_classes)

In [15]:
# training function
criterion = nn.CrossEntropyLoss()


def train(net, train_loader, valid_loader):
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)

    use_cuda = torch.cuda.is_available()
    if use_cuda:
        net = net.cuda()
        
    epoches = 5
    for epoch in range(epoches):
        net.train() 
        for i_batch, (images, labels) in enumerate(train_loader):
            if use_cuda:
                images = images.cuda()
                labels = labels.cuda()
            optimizer.zero_grad()
            outputs = net(images) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        print('Loss: %f '% (loss.item()))


    return net



In [16]:
train_net = train(net, data_loader, valid_loader)

Loss: 1.223983 
Loss: 0.872534 
Loss: 0.571535 
Loss: 0.268368 
Loss: 0.152818 


In [ ]:
torch.use_deterministic_algorithms(True)